---
# Alumno: Benjamín Alarcón 4945512
# Profesor: Dr. Diego Stalder
# Inteligencia Artificial Ing. Mecatrónica UNA
---

In [1]:
!pip install optuna

In [2]:
import pandas as pd
import numpy as np
import tensorflow as tf
import optuna
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import SparseCategoricalCrossentropy
tf.config.list_physical_devices('GPU')
tf.config.experimental.enable_op_determinism()
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import precision_score, recall_score, f1_score

In [3]:
SEED = 42

In [4]:
print(tf.config.list_physical_devices(device_type=None))

[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU'), PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [5]:
# Cargar el conjunto de datos
datos = pd.read_csv("Temp_Asu20092021.csv")

In [6]:
# Convertir la columna 'Fecha' al tipo datetime
datos['Fecha'] = pd.to_datetime(datos['Fecha'])
datos

,Unnamed: 0,Fecha,Temperatura,Humedad
0,0,2009-01-01 00:00:00,26.749483,69.609451
1,1,2009-01-01 03:00:00,25.956114,78.225166
2,2,2009-01-01 06:00:00,23.707357,93.551756
3,3,2009-01-01 09:00:00,23.726823,93.394817
4,4,2009-01-01 12:00:00,29.606379,72.155839
...,...,...,...,...
37979,37979,2021-12-31 09:00:00,21.719499,85.092387
37980,37980,2021-12-31 12:00:00,29.548918,53.780205
37981,37981,2021-12-31 15:00:00,40.026148,27.392646
37982,37982,2021-12-31 18:00:00,41.244712,19.955186


In [7]:
# Hacer que la 'Fecha' sea el índice
datos.set_index('Fecha', inplace=True)
datos = datos[datos.index.year >= 2019]
datos

,Unnamed: 0,Temperatura,Humedad
Fecha,,,
2019-01-01 00:00:00,29216,28.260936,66.311373
2019-01-01 03:00:00,29217,27.051418,68.882454
2019-01-01 06:00:00,29218,26.234018,72.660367
2019-01-01 09:00:00,29219,25.644533,74.647767
2019-01-01 12:00:00,29220,31.437436,55.215384
...,...,...,...
2021-12-31 09:00:00,37979,21.719499,85.092387
2021-12-31 12:00:00,37980,29.548918,53.780205
2021-12-31 15:00:00,37981,40.026148,27.392646


In [8]:
# Eliminar filas con valores faltantes
datos.dropna(inplace=True)

<ipython-input-8-e58a81f22bbb>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  datos.dropna(inplace=True)


In [9]:
# Para cambiar de muestreo de 3 horas a 1 hora interpolando valores
datos_interpolados = datos.resample('1H').interpolate(method='spline', order=3)
datos = datos_interpolados

In [10]:
# Eliminar filas con valores faltantes
datos.dropna(inplace=True)

In [11]:
# CALCULAR EL MÁXIMO DE CADA DÍA
max_temperaturas_diarias = datos['Temperatura'].resample('D').max()

In [12]:
# Aplicar corte de cuartil a las temperaturas máximas diarias
cuartiles = max_temperaturas_diarias.quantile([0.25, 0.5, 0.75])

In [13]:
# Definir categorías basadas en los cuartiles
umbral_frio = cuartiles[0.25]
umbral_bueno = cuartiles[0.5]
umbral_caliente = cuartiles[0.75]
print(cuartiles)
datos['Max_Temperatura_Dia'] = datos.groupby(datos.index.date)['Temperatura'].transform('max')

0.25    27.066809
0.50    31.893574
0.75    36.407532
Name: Temperatura, dtype: float64


In [14]:
# Asignar cada observación de temperatura a su categoría correspondiente
datos['Categoria_Temperatura'] = pd.cut(datos['Max_Temperatura_Dia'], bins=[-float('inf'), umbral_frio, umbral_bueno, umbral_caliente, float('inf')],
                                      labels=['Frío', 'Bueno', 'Caliente', 'Muy Caliente'])

In [15]:
# Desplazar para obtener la temperatura del siguiente día
datos['Temperatura_Siguiente_Dia'] = datos['Categoria_Temperatura'].shift(-24)

In [16]:
# USAR ONE HOT ENCODING
datos_codificados = pd.get_dummies(datos, columns=['Categoria_Temperatura'], drop_first=True)
datos_codificados

,Unnamed: 0,Temperatura,Humedad,Max_Temperatura_Dia,Temperatura_Siguiente_Dia,Categoria_Temperatura_Bueno,Categoria_Temperatura_Caliente,Categoria_Temperatura_Muy Caliente
Fecha,,,,,,,,
2019-01-01 00:00:00,29216.000000,28.260936,66.311373,37.639718,Muy Caliente,0,0,1
2019-01-01 01:00:00,29216.333333,28.223146,66.523607,37.639718,Muy Caliente,0,0,1
2019-01-01 02:00:00,29216.666667,27.908703,67.385359,37.639718,Muy Caliente,0,0,1
2019-01-01 03:00:00,29217.000000,27.051418,68.882454,37.639718,Muy Caliente,0,0,1
2019-01-01 04:00:00,29217.333333,26.729742,70.296842,37.639718,Muy Caliente,0,0,1
...,...,...,...,...,...,...,...,...
2021-12-31 17:00:00,37981.666667,41.562780,19.987211,41.562780,NaN,0,0,1
2021-12-31 18:00:00,37982.000000,41.244712,19.955186,41.562780,NaN,0,0,1
2021-12-31 19:00:00,37982.333333,41.059331,21.839127,41.562780,NaN,0,0,1


In [17]:
datos.drop(columns=['Unnamed: 0'], inplace=True)

In [20]:
# umbral
umbral_frio
datos_codificados['Categoria_Temperatura_Fria'] = datos_codificados['Temperatura'] < umbral_frio
datos_codificados

,Unnamed: 0,Temperatura,Humedad,Max_Temperatura_Dia,Temperatura_Siguiente_Dia,Categoria_Temperatura_Bueno,Categoria_Temperatura_Caliente,Categoria_Temperatura_Muy Caliente,Categoría_Temperatura_Fria,Categoria_Temperatura_Fria
Fecha,,,,,,,,,,
2019-01-01 00:00:00,29216.000000,28.260936,66.311373,37.639718,Muy Caliente,0,0,1,False,False
2019-01-01 01:00:00,29216.333333,28.223146,66.523607,37.639718,Muy Caliente,0,0,1,False,False
2019-01-01 02:00:00,29216.666667,27.908703,67.385359,37.639718,Muy Caliente,0,0,1,False,False
2019-01-01 03:00:00,29217.000000,27.051418,68.882454,37.639718,Muy Caliente,0,0,1,True,True
2019-01-01 04:00:00,29217.333333,26.729742,70.296842,37.639718,Muy Caliente,0,0,1,True,True
...,...,...,...,...,...,...,...,...,...,...
2021-12-31 17:00:00,37981.666667,41.562780,19.987211,41.562780,NaN,0,0,1,False,False
2021-12-31 18:00:00,37982.000000,41.244712,19.955186,41.562780,NaN,0,0,1,False,False
2021-12-31 19:00:00,37982.333333,41.059331,21.839127,41.562780,NaN,0,0,1,False,False


In [21]:
# Definir características y variable objetivo
X = datos_codificados[['Temperatura', 'Humedad', 'Max_Temperatura_Dia']]
y = datos_codificados['Categoria_Temperatura_Fria'] | datos_codificados['Categoria_Temperatura_Caliente']

In [22]:
# Normalizar características numéricas
scaler = StandardScaler()
X_escalado = scaler.fit_transform(X)

In [27]:
# Dividir datos en conjuntos de entrenamiento y validación
indices_entrenamiento = (datos.index.year <= 2019)
indices_validacion = (datos.index.year == 2020)

In [28]:
# Dividir datos en conjuntos de entrenamiento y validación
X_entrenamiento, X_validacion = X_escalado[indices_entrenamiento], X_escalado[indices_validacion]
y_entrenamiento, y_validacion = y[indices_entrenamiento], y[indices_validacion]

In [29]:
print(X_entrenamiento, X_validacion)
print(y_entrenamiento, y_validacion)

[[ 0.57526656  0.32631727  0.97821563]
 [ 0.57010223  0.33614881  0.97821563]
 [ 0.52713107  0.37606864  0.97821563]
 ...
 [-0.23754612  1.63594114  1.33146938]
 [-0.24498236  1.61285821  1.33146938]
 [-0.36474742  1.26608181  1.33146938]] [[-0.10074695  0.89877915  0.7359904 ]
 [-0.30489632  0.98566521  0.7359904 ]
 [-0.21415656  1.22925865  0.7359904 ]
 ...
 [ 0.67149654  0.39911063 -0.10387645]
 [ 0.35946704  0.53925582 -0.10387645]
 [ 0.29630772  0.66381963 -0.10387645]]
Fecha
2019-01-01 00:00:00    False
2019-01-01 01:00:00    False
2019-01-01 02:00:00    False
2019-01-01 03:00:00     True
2019-01-01 04:00:00     True
                       ...  
2019-12-31 19:00:00    False
2019-12-31 20:00:00    False
2019-12-31 21:00:00     True
2019-12-31 22:00:00     True
2019-12-31 23:00:00     True
Freq: H, Length: 8760, dtype: bool Fecha
2020-01-01 00:00:00     True
2020-01-01 01:00:00     True
2020-01-01 02:00:00     True
2020-01-01 03:00:00     True
2020-01-01 04:00:00     True
         

In [36]:
def objetivo(trial):
    # Definir hiperparámetros a ajustar
    tf.keras.utils.set_random_seed(SEED)

    # Probar diferentes números de unidades en las capas
    num_unidades = trial.suggest_categorical('num_unidades', [32, 64, 128])
    # Probar diferentes tasas de aprendizaje
    tasa_aprendizaje = trial.suggest_float('tasa_aprendizaje', 1e-4, 1e-2, log=True)
    # Probar diferentes números de capas
    num_capas = trial.suggest_int('num_capas', 1, 3)
    # Opcional: probar la inclusión de capas de Dropout
    dropout_rate = trial.suggest_float('dropout_rate', 0.0, 0.5)

    # Definir la arquitectura del modelo
    modelo = Sequential()
    modelo.add(Dense(num_unidades, activation='relu', input_shape=(X.shape[1],)))
    # Agregar capas adicionales basadas en num_capas
    for _ in range(num_capas - 1):
        modelo.add(Dense(num_unidades, activation='relu'))
        # Incluir Dropout si se sugiere
        if dropout_rate > 0.0:
            modelo.add(tf.keras.layers.Dropout(dropout_rate))
    # La capa de salida
    modelo.add(Dense(1, activation='sigmoid'))  # Para clasificación binaria

    # Compilar el modelo
    modelo.compile(optimizer=Adam(learning_rate=tasa_aprendizaje),
                   loss='binary_crossentropy',  # Ajustado para clasificación binaria
                   metrics=['accuracy'])

    # Entrenar el modelo
    modelo.fit(X_entrenamiento, y_entrenamiento, validation_data=(X_validacion, y_validacion),
               epochs=trial.suggest_int('epochs', 50, 100),  # Probar diferentes números de épocas
               batch_size=trial.suggest_categorical('batch_size', [64, 128, 256]),  # Probar diferentes tamaños de batch
               verbose=0, shuffle=True)  # Usar shuffle=True es generalmente recomendado

    # Evaluar el modelo en el conjunto de validación
    _, val_acc = modelo.evaluate(X_validacion, y_validacion, verbose=0)

    return val_acc


In [38]:
# Realizar la optimización de hiperparámetros usando Optuna
import keras
estudio = optuna.create_study(direction='maximize') # cambiar si se modificar la metrica de la funcion objetivo, se puede hacer un problema multiobjetivo tambien
estudio.optimize(objetivo, n_trials=10) # probar mas trials una vez que se tenga el script funcionando, minimo 50, evitar variar muchos parametros al mismo tiempo

[I 2024-03-23 23:03:55,982] A new study created in memory with name: no-name-c4536978-aace-4cb2-b25c-ff3926c1ff50
<ipython-input-36-43e2de69742a>:12: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float instead.
  dropout_rate = trial.suggest_uniform('dropout_rate', 0.0, 0.5)
[I 2024-03-23 23:04:28,121] Trial 0 finished with value: 0.9683515429496765 and parameters: {'num_unidades': 64, 'tasa_aprendizaje': 0.0005275076338998948, 'num_capas': 1, 'dropout_rate': 0.3169799392990271, 'epochs': 92, 'batch_size': 128}. Best is trial 0 with value: 0.9683515429496765.
[I 2024-03-23 23:05:51,105] Trial 1 finished with value: 0.9812158346176147 and parameters: {'num_unidades': 64, 'tasa_aprendizaje': 0.0007095104287166017, 'num_capas': 1, 'dropout_rate': 0.3632109033711507, 'epochs': 85, 'batch_size': 64}. Best is trial 1 with value: 0.9812158346176147.
[I 2024-03-23 23:06

In [39]:
# Obtener los mejores hiperparámetros
mejor_num_capas = estudio.best_params['num_capas']
mejor_num_unidades = estudio.best_params['num_unidades']
mejor_tasa_aprendizaje = estudio.best_params['tasa_aprendizaje']

In [40]:
print("Mejores Hiperparámetros:")
print("Número de Capas:", mejor_num_capas)
print("Número de Unidades:", mejor_num_unidades)
print("Tasa de Aprendizaje:", mejor_tasa_aprendizaje)

Mejores Hiperparámetros:
Número de Capas: 3
Número de Unidades: 64
Tasa de Aprendizaje: 0.004532257726821485


In [41]:
# Entrenar el modelo final usando los mejores hiperparámetros
modelo_final = Sequential()
modelo_final.add(Dense(mejor_num_unidades, activation='relu', input_shape=(X_entrenamiento.shape[1],)))
for _ in range(mejor_num_capas - 1):
    modelo_final.add(Dense(mejor_num_unidades, activation='relu'))
modelo_final.add(Dense(4, activation='softmax'))

modelo_final.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=mejor_tasa_aprendizaje),
                     loss='categorical_crossentropy',
                     metrics=['accuracy'])

In [49]:
# Dividir datos en conjuntos de entrenamiento y prueba
indices_entrenamiento_final = (datos.index.year <= 2020)
indices_prueba_final = (datos.index.year >= 2021)

In [50]:
# Dividir datos en conjuntos de entrenamiento y prueba
X_entrenamiento, X_prueba = X.loc[indices_entrenamiento_final], X.loc[indices_prueba_final]
y_entrenamiento, y_prueba = y.loc[indices_entrenamiento_final], y.loc[indices_prueba_final]

In [54]:
y_entrenamiento_one_hot = tf.keras.utils.to_categorical(y_entrenamiento, num_classes=4)
y_prueba_one_hot = tf.keras.utils.to_categorical(y_prueba, num_classes=4)

In [55]:
modelo_final.fit(X_entrenamiento, y_entrenamiento_one_hot, validation_data=(X_prueba, y_prueba_one_hot), epochs=50, batch_size=256, verbose=1, shuffle=False)

Epoch 1/50


/usr/local/lib/python3.10/dist-packages/tensorflow/python/data/ops/structured_function.py:258: UserWarning: Even though the `tf.config.experimental_run_functions_eagerly` option is set, this option does not apply to tf.data functions. To force eager execution of tf.data functions, please use `tf.data.experimental.enable_debug_mode()`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/tensorflow/python/data/ops/structured_function.py:258: UserWarning: Even though the `tf.config.experimental_run_functions_eagerly` option is set, this option does not apply to tf.data functions. To force eager execution of tf.data functions, please use `tf.data.experimental.enable_debug_mode()`.
  warnings.warn(


69/69 [==============================] - ETA: 0s - loss: 0.5729 - accuracy: 0.8043

/usr/local/lib/python3.10/dist-packages/tensorflow/python/data/ops/structured_function.py:258: UserWarning: Even though the `tf.config.experimental_run_functions_eagerly` option is set, this option does not apply to tf.data functions. To force eager execution of tf.data functions, please use `tf.data.experimental.enable_debug_mode()`.
  warnings.warn(


69/69 [==============================] - 64s 918ms/step - loss: 0.5729 - accuracy: 0.8043 - val_loss: 0.4759 - val_accuracy: 0.7772
Epoch 2/50
69/69 [==============================] - 3s 47ms/step - loss: 0.4110 - accuracy: 0.8170 - val_loss: 0.3547 - val_accuracy: 0.8135
Epoch 3/50
69/69 [==============================] - 4s 55ms/step - loss: 0.3814 - accuracy: 0.8229 - val_loss: 0.4237 - val_accuracy: 0.7682
Epoch 4/50
69/69 [==============================] - 3s 46ms/step - loss: 0.3851 - accuracy: 0.8178 - val_loss: 0.4028 - val_accuracy: 0.7839
Epoch 5/50
69/69 [==============================] - 3s 43ms/step - loss: 0.3709 - accuracy: 0.8073 - val_loss: 0.4423 - val_accuracy: 0.7176
Epoch 6/50
69/69 [==============================] - 3s 50ms/step - loss: 0.3891 - accuracy: 0.8076 - val_loss: 0.3503 - val_accuracy: 0.8250
Epoch 7/50
69/69 [==============================] - 3s 46ms/step - loss: 0.3951 - accuracy: 0.8019 - val_loss: 0.3669 - val_accuracy: 0.7755
Epoch 8/50
69/69 [====

In [56]:
# Ahora realiza la evaluación con las etiquetas en formato one-hot.
_, val_acc = modelo_final.evaluate(X_prueba, y_prueba_one_hot, verbose=0)
print(val_acc)


/usr/local/lib/python3.10/dist-packages/tensorflow/python/data/ops/structured_function.py:258: UserWarning: Even though the `tf.config.experimental_run_functions_eagerly` option is set, this option does not apply to tf.data functions. To force eager execution of tf.data functions, please use `tf.data.experimental.enable_debug_mode()`.
  warnings.warn(


0.872231125831604


In [57]:
# Ejemplo de como calcular las otras metricas que podrias utilizarse como funcion objetivo para la optimizacion de hiperparametros en lugar del acc
y_pred = modelo_final.predict(X_prueba)
y_pred_classes = np.argmax(y_pred, axis=1)  # Convertir las predicciones en clases

 24/274 [=>............................] - ETA: 1s

/usr/local/lib/python3.10/dist-packages/tensorflow/python/data/ops/structured_function.py:258: UserWarning: Even though the `tf.config.experimental_run_functions_eagerly` option is set, this option does not apply to tf.data functions. To force eager execution of tf.data functions, please use `tf.data.experimental.enable_debug_mode()`.
  warnings.warn(


274/274 [==============================] - 1s 4ms/step


In [58]:
y_pred_classes = np.argmax(y_pred, axis=1)

precision = precision_score(y_prueba, y_pred_classes, average='weighted')
recall = recall_score(y_prueba, y_pred_classes, average='weighted')
f1 = f1_score(y_prueba, y_pred_classes, average='weighted')

print("Precision:", precision)
print("Recall:", recall)
print("F1 Score:", f1)

Precision: 0.8618770863749612
Recall: 0.8722311029915506
F1 Score: 0.8636800234980324
